In [9]:
# Updating pip
%pip install --upgrade pip

  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Import required libraries
from neuralprophet import NeuralProphet, set_log_level
import pandas as pd

# Disable logging messages
set_log_level("ERROR")

# Load and preprocess data
dfa = pd.read_csv("aggregated_daily_sales.csv")
# Select the needed columns from the dataframe
df = dfa[["ds", "y"]]
# Drop duplicates and ensure "ds" column is datetime format
df = df.drop_duplicates(subset="ds").assign(ds=lambda x: pd.to_datetime(x["ds"]))

# Model configuration
confidence_level = 0.9
boundaries = round((1 - confidence_level) / 2, 2)
quantiles = [boundaries, 1 - boundaries]

# Initialize model with quantiles and set plotting backend to plotly
m = NeuralProphet(quantiles=quantiles)
m.set_plotting_backend("plotly")

# Add country holidays and custom events
m.add_country_holidays("EC")
m.add_events(["earthquake"])

# Prepare events dataframe
earthquake_dates = ["2016-04-17", "2016-04-18", "2016-04-19", "2016-04-20"]
df_events = pd.DataFrame(
    {"event": "earthquake", "ds": pd.to_datetime(earthquake_dates)}
)

df_all = m.create_df_with_events(df, df_events)

# Fit the model and perform forecasting
metrics = m.fit(df_all)
df_future = m.make_future_dataframe(df_all, periods=365, n_historic_predictions=True)
forecast = m.predict(df_future)

# plotting


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\lenovo_laptop\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
  File "C:\Users\lenovo_laptop\AppData\Local\Temp\ipykernel_436\3970771755.py", line 2, in <module>
    from neuralprophet import NeuralProphet, set_log_level
ModuleNotFoundError: No module named 'neuralprophet'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\lenovo_laptop\anaconda3\envs\myenv\lib\site-packages\pygments\styles\__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\lenovo_laptop\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
  File "c:\Users\lenovo_laptop\anaconda3\envs\myenv\lib\site-packages\IPython\core\ultrat

In [ ]:
# Import required libraries
from neuralprophet import NeuralProphet, set_log_level
import pandas as pd

# Disable logging messages
set_log_level("ERROR")

# Load and preprocess data
dfa = pd.read_csv("aggregated_daily_sales.csv")
# Select the needed columns from the dataframe
df = dfa[["ds", "y"]]
# Drop duplicates and ensure "ds" column is datetime format
df = df.drop_duplicates(subset="ds").assign(ds=lambda x: pd.to_datetime(x["ds"]))

# Model configuration
confidence_level = 0.9
boundaries = round((1 - confidence_level) / 2, 2)
quantiles = [boundaries, 1 - boundaries]

# Initialize model with quantiles and set plotting backend to plotly
m = NeuralProphet(quantiles=quantiles)
m.set_plotting_backend("plotly")

# Add country holidays and custom events
m.add_country_holidays("EC")
m.add_events(["earthquake"])

# Prepare events dataframe
earthquake_dates = ["2016-04-17", "2016-04-18", "2016-04-19", "2016-04-20"]
df_events = pd.DataFrame(
    {"event": "earthquake", "ds": pd.to_datetime(earthquake_dates)}
)

df_all = m.create_df_with_events(df, df_events)

# Fit the model and perform forecasting
metrics = m.fit(df_all)
df_future = m.make_future_dataframe(df_all, periods=365, n_historic_predictions=True)
forecast = m.predict(df_future)

# plotting


In [ ]:
import numpy as np
from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import DistributionLoss
from neuralforecast import NeuralForecast
import matplotlib.pyplot as plt
import pandas as pd

# Process df DataFrame to obtain dataset for the whole period
residuals_df = df.copy()
residuals_df["residuals"] = df["y"] - forecast['yhat1'] # Adjust to use forecasted values for residuals calculation
residuals_df["unique_id"] = "1"
residuals_df = residuals_df[["ds", "residuals", "unique_id"]]

# Instantiate the NBEATSx model with pre-defined parameters
model = NBEATSx(
    h=99,
    input_size=24,
    loss=DistributionLoss(distribution="Normal", level=[80, 90]),
    scaler_type="robust",
    max_steps=200,
    val_check_steps=100,
    batch_size=32,
    windows_batch_size=1024,
    random_seed=1,
)
nf = NeuralForecast(models=[model], freq="D")

# Fit and predict using NeuralForecast
nf.fit(residuals_df, id_col="unique_id", time_col="ds", target_col="residuals")
y_hat = nf.predict(residuals_df)

# Visualization of residuals and forecast
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(residuals_df["ds"], residuals_df["residuals"], label="Residuals")
ax.plot(y_hat["ds"], y_hat["NBEATSx"], label="Forecast", color="#7B3841")
ax.legend()
ax.set_xlabel("Timestamp [t]")
ax.set_ylabel("Residuals [t]")
plt.show()